In [ ]:
import requests
from bs4 import BeautifulSoup
import requests
import re
import getopt,sys
import time
import pickle

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
url = 'http://www.sports-reference.com/cfb/schools/utah'
data= {}
data['years'] = {}
data['opponents'] = {}

start = time.time()
raw_html = requests.get(url, headers=header)
parsed = BeautifulSoup(raw_html.text, 'html.parser')
utah = parsed.find(id='div_utah')
utah_table = utah.find('tbody')
print('Getting years')
for yearRow in utah_table.find_all('tr', {'class': False}): 
    sys.stdout.write('.')
    sys.stdout.flush()
    yearCol = yearRow.find('td',{'data-stat': 'year_id'})
    year = yearCol.a.text
    data['years'][year] = {}
    data['years'][year]['conference'] = yearRow.find('td', {'data-stat': 'conf_id'}).a.text
    data['years'][year]['wins'] = yearRow.find('td', {'data-stat': 'wins'}).text
    data['years'][year]['losses'] = yearRow.find('td', {'data-stat': 'losses'}).text

print('\n')
print('Getting schedules')
for year, yearData in data['years'].items():
    sys.stdout.write('.')
    sys.stdout.flush()
#    print(year)
    yearUrl = url+'/'+year+'-schedule.html'
    yearRaw = requests.get(yearUrl, headers=header)
    yearParsed = BeautifulSoup(yearRaw.text, 'html.parser')
    data['years'][year]['schedule'] = []
    tb = yearParsed.find(id='div_schedule').find(id='schedule').thead.find_next_sibling()
    for gameData in tb.find_all('tr'):
        # Parse game
        game = {}
        
        # Get date
        gameDate = gameData.find('td', {'data-stat': 'date_game'}).text
        game['date'] = gameDate
        
        # Get opponent
        opponentRaw = gameData.find('td', {'data-stat': 'opp_name'}).text.replace('\xa0', ' ')
        opponent = re.sub(r'\(\d+\)', '',opponentRaw).strip()
        game['opponent'] = opponent
        
        # Get result
        gameResult = gameData.find('td', {'data-stat': 'game_result'}).text
        game['result'] = gameResult
        
        # Add to list
        data['years'][year]['schedule'].append(game)
        
        # Add to opponent table
        if not opponent in data['opponents']:
 #           print('Adding: '+opponent)
            data['opponents'][opponent] = []
            
        # Get points
        points = gameData.find('td', {'data-stat': 'points'}).text
        oppPoints = gameData.find('td', {'data-stat': 'opp_points'}).text
        oppGame = {}
        oppGame['date'] = gameDate
        oppGame['result'] = gameResult
        oppGame['points'] = points
        oppGame['opp_points'] = oppPoints
        data['opponents'][opponent].append(oppGame)
    
end = time.time()
print('')
print(end - start)


sys.setrecursionlimit(10000)
pickle.dump(data, open('utah_all.p', 'wb'))
print('done')
print(data['years']['2008'])
print(data['opponents']['Brigham Young'])


In [ ]:
import sys
import pickle

alldata = pickle.load(open('utah_all.p', 'rb'))
byu = alldata['opponents']['Brigham Young']
stanford = alldata['opponents']['Stanford']

print('BYU: '+str(len(byu)))
print('Stanford: '+str(len(stanford)))
for game in stanford:
    print(game)

In [ ]:
import re

byu = alldata['opponents']['Brigham Young']
print('BYU: '+str(len(byu)))

game = byu[0]
gameDate = game['date']
gameYear = re.search('.*, (\d+)',gameDate)
print(gameYear.group(1))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import FuncFormatter



In [107]:
import requests
from bs4 import BeautifulSoup as bs4
import re

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}

for sch in range(115,420): 
#for sch in range(115,120): 
    sched = str(sch)
    url = 'https://www.utahutes.com/schedule.aspx?schedule='+sched
    #print(url)
    page=requests.get(url, headers=header)
    parsed=bs4(page.text, 'html.parser')
    scheduleTable = parsed.find('article', {'class', 'sidearm-schedule'})
    scheduleType = scheduleTable.find('h2')
    if 'Football' in scheduleType.text:
        year = re.search('([0-9]+).*',scheduleType.text).group(1)
        print(str(sch)+' - '+year)
        for game in scheduleTable.find_all('li', {'class': 'sidearm-schedule-game'}):
            gameRow = game.find('div', {'class': 'sidearm-schedule-game-row'})
            opponent = gameRow.find('div', {'class': 'sidearm-schedule-game-opponent'})
            details = opponent.find('div', {'class': 'sidearm-schedule-game-opponent-details'})
            gameDateRaw = details.find('div', {'class': 'sidearm-schedule-game-opponent-date'}).find('span')
            gameDate = re.search('([a-zA-Z]+ [0-9]+) .*', gameDateRaw.text.strip())
            print('  '+gameDate.group(1))
            opp = details.find('div', {'class': 'sidearm-schedule-game-opponent-text'})
            oppName = opp.find('span', {'class': 'sidearm-schedule-game-opponent-name'})
            print('  '+oppName.text.strip())
            print('')
    

115 - 1892
  Nov 25
  Utah State

116 - 1894
  Oct 4
  Salt Lake City High

  Nov 3
  Salt Lake City YMCA

  Nov 29
  Salt Lake City High

117 - 1895
  Nov 28
  Salt Lake City YMCA

  Apr 6
  BYU

118 - 1896
  Nov 14
  BYU

  Nov 26
  Salt Lake City Crescents

  Dec 5
  BYU

  Dec 25
  Salt Lake City High

119 - 1897
  Oct 23
  Salt Lake City YMCA

  Nov 1
  LDS College

  Nov 5
  Westminster College (UT)

  Nov 25
  Salt Lake City YMCA

  Dec 4
  BYU

  Dec 18
  BYU

120 - 1898
  Nov 14
  Salt Lake City High

  Nov 24
  BYU

  Dec 12
  All Hallows College

121 - 1899
  Nov 3
  9th Calvary

  Nov 13
  9th Calvary

  Nov 25
  Salt Lake City High

122 - 1900
  Nov 10
  All Hallows College

  Nov 17
  Utah State

  Nov 24
  Salt Lake City YMCA

123 - 1901
  Oct 19
  Ogden High School

  Oct 26
  All Hallows College

  Nov 9
  Utah State

  Nov 28
  Nevada

124 - 1902
  Sep 26
  Fort Douglas (UT)

  Oct 4
  Ogden High School

  Oct 11
  Salt Lake City High

  Oct 18
  Fort Douglas (UT)

  

165 - 1944
  Sep 30
  Idaho State

  Oct 6
  Denver

  Oct 14
  Colorado

  Oct 21
  Idaho State

  Oct 28
  Nevada

  Nov 4
  Denver

  Nov 11
  Colorado College

  Nov 23
  Utah State

166 - 1945
  Sep 29
  Nevada

  Oct 6
  Colorado

  Oct 12
  Denver

  Oct 20
  Oklahoma State

  Oct 27
  Colorado State

  Nov 3
  Denver

  Nov 10
  New Mexico

  Nov 22
  Denver

167 - 1946
  Sep 28
  New Mexico

  Oct 5
  Arizona

  Oct 12
  BYU

  Oct 19
  Denver

  Oct 26
  Wyoming

  Nov 2
  Colorado

  Nov 9
  Colorado State

  Nov 16
  San Francisco

  Nov 28
  Utah State

  Dec 25
  Hawaii All-Stars

  Jan 1
  Hawai'i

168 - 1947
  Sep 27
  Oregon State

  Oct 4
  Hawai'i

  Oct 11
  BYU

  Oct 18
  Denver

  Oct 25
  Wyoming

  Nov 1
  Colorado

  Nov 8
  Colorado State

  Nov 15
  Idaho

  Nov 27
  Utah State

  Dec 6
  Arizona

169 - 1948
  Sep 17
  USC

  Sep 25
  Idaho

  Oct 2
  Arizona

  Oct 9
  BYU

  Oct 16
  Denver

  Oct 23
  Wyoming

  Oct 30
  Colorado

  Nov 6
  Colorado State

202 - 1981
  Sep 5
  Utah State

  Sep 12
  Arizona State

  Sep 19
  Portland State

  Sep 26
  Northwestern

  Oct 2
  UTEP

  Oct 17
  Colorado State

  Oct 24
  UNLV

  Oct 31
  San Diego State

  Nov 7
  New Mexico

  Nov 14
  Wyoming

  Nov 21
  BYU

203 - 1982
  Sep 4
  Montana State

  Sep 11
  Arizona State

  Sep 18
  Texas

  Oct 1
  Colorado State

  Oct 9
  Hawai'i

  Oct 16
  UNLV

  Oct 23
  Wyoming

  Oct 30
  San Diego State

  Nov 6
  Utah State

  Nov 13
  UTEP

  Nov 20
  BYU

204 - 1983
  Sep 3
  New Mexico

  Sep 10
  Arizona

  Sep 17
  San Diego State

  Oct 1
  Colorado State

  Oct 8
  UTEP

  Oct 9
  Hawai'i

  Oct 15
  Wyoming

  Oct 22
  Air Force

  Nov 5
  Cal State Fullerton

  Nov 12
  Utah State

  Nov 19
  Utah State

205 - 1984
  Sep 1
  Weber State

  Sep 8
  Washington State

  Sep 15
  Tennessee

  Sep 22
  Air Force

  Sep 29
  Wyoming

  Oct 5
  New Mexico

  Oct 13
  San Diego State

  Oct 20
  Hawai'i

  Oct 27
  UTEP

  Nov 3
  Colorado State

232 - 2011
  Sep 1
  Montana State

  Sep 10
  USC

  Sep 17
  BYU

  Oct 1
  Washington

  Oct 8
  Arizona State

  Oct 15
  Pittsburgh

  Oct 22
  California

  Oct 29
  Oregon State

  Nov 5
  Arizona

  Nov 12
  UCLA

  Nov 19
  Washington State

  Nov 25
  Colorado

  Dec 31
  Georgia Tech

233 - 2012
  Aug 30
  Northern Colorado

  Sep 7
  Utah State

  Sep 15
  Brigham Young

  Sep 22
  Arizona State

  Oct 4
  USC

  Oct 13
  UCLA

  Oct 20
  Oregon State

  Oct 27
  California

  Nov 3
  Washington State

  Nov 10
  Washington

  Nov 17
  Arizona

  Nov 23
  Colorado

234 - 2013
  Aug 29
  Utah State

  Sep 7
  Weber State

  Sep 14
  Oregon State (Homecoming)

  Sep 21
  BYU

  Oct 3
  UCLA

  Oct 12
  Stanford

  Oct 19
  Arizona

  Oct 26
  USC

  Nov 9
  Arizona State

  Nov 16
  Oregon

  Nov 23
  Washington State

  Nov 30
  Colorado

235 - 2014
  Aug 28
  Idaho State

  Sep 6
  Fresno State

  Sep 20
  Michigan

  Sep 27
  Washington State

  Oct 4
  UCLA

  Oct 16
  Or

AttributeError: 'NoneType' object has no attribute 'find'

In [24]:
foo = 'the foobar is in'
if 'foo' in foo:
    print('yes')
else:
    print('no')

yes
